# Recipe 3: Anomaly Detection
## Overview 

In the previous recipe, `2-interpretability.ipynb` we demonstrated some of the core interpretability capabilities and tools available in Howso Engine. In this recipe, we will use some of those methods to demonstrate a common use case, anomaly detection. Howso Engine  can be used to identify anomalous Cases in a dataset and also explain why cases those may be anomalous. Furthermore, we can leverage other tools we learned in previous recipes, such influential and boundary Cases, to provide additional context.

Finding anomalous cases has value in both modeling building and data exploration. For example, as we continue to build a suitable model for the `Adult` datset, detecting anomalies can give us insight into whether we need to clean the dataset to reduce false signals. Possible sources of anomalies could be Cases that are erroneous or falsified.   



## Recipe Goals: 
This recipe demonstrates how to use some of the tools learned in `2-interpretability.ipynb`, as well as demonstrate some additional metrics to detect anomalies. As we continue to build our Trainee for the `Adult` dataset, we want to be confident that our training data is as clean as possible.

# Section 1: Train and Analyze



### 1. Load data

Our example dataset for this recipe continues to be the well known `Adult` dataset. This dataset consists of 14 Context Features and 1 Action Feature. The Action Feature in this version of the `Adult` dataset has been renamed to `target` and it takes the form of a binary indicator for whether a person in the data makes over $50,000/year (*target*=1) or less (*target*=0).

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pmlb import fetch_data
import seaborn as sns

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes
from howso.visuals import plot_anomalies

In [2]:
df = fetch_data('adult', local_cache_dir="data/adult")

# Subsample the data to ensure the example runs quickly
df = df.sample(1000, random_state=0)

df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
38113,41.0,4,151856.0,11,9.0,2,11,0,4,1,0.0,0.0,40.0,39,1
39214,57.0,6,87584.0,10,16.0,0,10,1,4,0,0.0,0.0,25.0,39,1
44248,31.0,2,220669.0,9,13.0,4,10,3,4,0,6849.0,0.0,40.0,39,1
10283,55.0,4,171355.0,8,11.0,2,7,0,4,1,0.0,0.0,20.0,39,1
26724,59.0,6,148626.0,0,6.0,2,5,0,4,1,0.0,0.0,40.0,39,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33804,29.0,4,166220.0,9,13.0,4,12,3,4,0,0.0,0.0,50.0,39,1
4721,60.0,0,204486.0,9,13.0,2,0,0,4,1,0.0,0.0,8.0,39,0
40113,48.0,2,93449.0,14,15.0,2,10,0,1,1,99999.0,0.0,40.0,30,0
17827,25.0,4,114838.0,14,15.0,4,10,1,4,0,0.0,0.0,8.0,22,1


### 2. Train Trainee

In this section we will perform all of the steps needed to train Howso Engine's Trainee.

While we preivously separated the Action and Context Features, we will leave them together for anomaly detection to incorpate all the features in a targetless flow.

In [3]:
# Infer features attributes
features = infer_feature_attributes(df)

# Create the Trainee
t = Trainee(
    features=features,
    overwrite_existing=True
)

t.train(df)

# Targetless Analysis
t.analyze()

# Section 2. Anomaly Identification

We will use `Familiarity Conviction` and `Distance Contribution` to identify potentially anomalous cases.`Familiarity Conviction` is metric for describing surprisal of individual Cases in the Trainee relative to other Cases. In other words, Familiarity Conviction evaluates each data point in the Trainee and assigns a surprisal score indicating how similar or different each point is to the remainder of the data. Howso Engine’s interpretability tools also enable us to understand why these cases may be anomalous and whether they are outliers and inliers. 

**`Definitions`:**

**`Familiarity Conviction`:** How confident or familiar the Trainee is about some data point on which it has been trained, as determined by the KL Divergence of the effect of the particular data point on the probability density function of the overall data. The lower the Familiarity Conviction value, the less familiar the Trainee is with the data point. Thus, a Familiarity Conviction of 0.01 corresponds to the Trainee believing a data point is unusual, while a Familiarity Conviction of 2 corresponds to the Trainee believing a data point is fairly familiar. Low values can also be used to determine when further training is needed to improve the Trainee’s ability to provide accurate results. 

An example is if there is a dataset that comprises four data points, such as the corners of a square on a grid. A new data point that lies inside the square, will return high `Familiarity Conviction` (low surprisal) as it fits within the data scheme. However, another data point that lies diagonally from the uppermost corner will have a low `Familiarity Conviction` (high surprisal) because it does not fit in the square, like the other points.

**`Distance Contribution`**: The expected total surprisal contribution for a Case. That is, the amount of distance (or knowledge) a Case adds to the Trainee, where the distance is measured in surprisal.

### Step 1. React

The first step is to use `react_into_features` and select the metrics we want to return.

In [4]:
# Store the familiarity conviction, this will be used to identify anomalous cases
t.react_into_features(
    familiarity_conviction_addition=True,
    distance_contribution=True
)

stored_convictions = t.get_cases(
    session=t.active_session,
    features=df.columns.tolist() + ['familiarity_conviction_addition','.session_training_index', '.session', 'distance_contribution']
)

### Step 2. Determine Threshold

There is no absolute threshold for what is considered to be anomalous, as anomalous behavior is a measure of surprisal. Thus, this is a lever that the user can change depending on the Trainee. The threshold should be less than 1, which is average surprisal.

We first use `familiarity_conviction_addition` to find anomalies, and then use the average `distance_contribution` to determine what type of anomaly the case is (e.g., inlier vs. outlier).

In [5]:
# Threshold to determine which cases will be deemed anomalous
convict_threshold = 0.75

# Extract the anomalous cases
low_convicts = stored_convictions[
    stored_convictions['familiarity_conviction_addition'] <= convict_threshold 
].sort_values('familiarity_conviction_addition', ascending=True)

# Average distance contribution will be used to determine if a case is an outlier or inlier
average_dist_contribution = low_convicts['distance_contribution'].mean()

# A case with distance contribution greater than average will be tagged as outlier, and vise versa for inliers
cat = [
    'inlier' if d < average_dist_contribution else 
    'outlier' for d in low_convicts['distance_contribution']
]
low_convicts['category'] = cat

# Section 3. Anomaly Inspection



### Step 1. Outliers

Typically, when we think of anomalies, we think of outliers or cases that are significantly different that the rest of the data, whether erroneous or not. Preprocessing in machine learning often involves pruning these data points as they can add undesired noise to a model. 

The Case Feature Residual Conviction will be used to understand why each Case was anomalous. There are two types of Reature Residual Convictions: `global_case_feature_residual_convictions` and `local_case_feature_residual_convictions`.

In our example use case where the `Adult` dataset is used to determine projected salary for loan applications, we want to eliminate any erroneous outliers if possible. We may also want to detect substituted values, as often times datasets may substitute special values with nominal integers. For example, blank values in an application are often coded with high integers like 99999 which can introduce noise in the dataset.

**`Definitions`:**

**`global_case_feature_residual_convictions`** : A Case's Feature Residual Convictions for the global model. This is calculated using the expected feature Residuals, divided by the feature Residuals of the Case to obtain the Convictions. The global feature Residuals are used as the expected value.

**`local_case_feature_residual_convictions`** : A Case's Feature Residual Convictions for the local model. This is calculated using the expected feature Residuals, divided by the feature Residuals of the Case to obtain the Convictions. The local feature residuals are used as the expected value.

#### Step 1a. Outlier Extraction

In [6]:
# Extract the outliers cases
outliers = low_convicts[low_convicts['category'] == 'outlier'].reset_index(drop=True)
outliers.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,category
0,28.0,4,126060.0,11,9.0,2,1,5,4,0,99999.0,0.0,36.0,39,0,0.079562,627,cd9d5349-22d4-43a9-aebb-68419083a4b1,30.557225,outlier
1,54.0,6,269068.0,14,15.0,2,10,0,1,1,99999.0,0.0,50.0,30,0,0.090035,556,cd9d5349-22d4-43a9-aebb-68419083a4b1,26.299004,outlier
2,48.0,2,93449.0,14,15.0,2,10,0,1,1,99999.0,0.0,40.0,30,0,0.092224,997,cd9d5349-22d4-43a9-aebb-68419083a4b1,25.532120,outlier


#### Step 1b. Outlier Evaluation

From the above results, we can see which cases may be anomalous, however just knowing whether a case is potentially an anomaly doesn't give us all the information we need to make an informed decision. If we are deciding whether to prune a Case from our training set, often we only want to prune erroneous values. Even if a Case is an anomaly in terms of distance or value, if it is valid we may still leave it in as it represents a legitimate data point. 

By looking at the `Global Case Feature Residual Convictions`, we can gain insight as to `why` a Case was anomalous.

In [7]:
# We add a few extra metrics for subsequent examples in this notebook
# Get the case_feature_residual_convictions, influential_cases and boundary_cases
details = {
    'robust_computation':True,
    'boundary_cases':True, 
    'num_boundary_cases':3, 
    'influential_cases':True, 
    'global_case_feature_residual_convictions':True, 
    'local_case_feature_residual_convictions':True
}

# Specify outlier cases
outliers_indices = outliers[['.session', '.session_training_index']].values

# React to get the details of each case
results = t.react(
    case_indices=outliers_indices, 
    preserve_feature_values=df.columns.tolist(), 
    leave_case_out=True, 
    details=details
)

# Extract the global and local case feature residual convictions
outlier_global_case_feature_residual_convictions = pd.DataFrame(
    results['explanation']['global_case_feature_residual_convictions']
)[df.columns.tolist()]

outlier_local_case_feature_residual_convictions = pd.DataFrame(
    results['explanation']['local_case_feature_residual_convictions']
)[df.columns.tolist()]

In [8]:
plot_anomalies(outliers, outlier_global_case_feature_residual_convictions)

#### Step 1c. Outlier Inspection

From this chart we can see that these Cases were flaggged as anomalous largely because of their `capital-gain` values. Since our target variable is whether someone makes over $50,000 in salary, people with such large capital gains making less than $50,000 seems odd. In addition, values such as 99999 often indicate some sort of nominal value that may represent something other than the actual capital-gain, such as if a person did not answer. 

With this information, we can choose the appropriate action, whether it is recoding capital-gains, removing the Cases, leaving the Cases out, etc...

### Step 2. Inliers

While we generally only think of outliers when it comes to anomalies, inliers are another, more discrete, form of anomaly. Inliers are the opposite of outliers, as inliers are Cases that are too `similiar` to other points. A real world hypothetical case of an inlier would be someone submitting a false transaction date in which they tried to make the data look real by making it very similar to existing data. 

In our case, it may be someone in a bank who is internally falsifying applications in order to try to alter the prediction model.

Just like outliers, `distance_contribution` is used to detect inliers from the anomalies detected through the use of `familiarity_conviction_addition`, except now we are looking at Cases below a theshold which indicates similarity to other points.


#### Step 2a. Inlier Extraction

In [9]:
# Extract the inlier cases
inliers = low_convicts[low_convicts['category'] == 'inlier'].reset_index(drop=True)
inliers.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,category
0,30.0,2,181091.0,9,13.0,2,11,0,4,1,0.0,0.0,40.0,39,1,0.140530,624,cd9d5349-22d4-43a9-aebb-68419083a4b1,1.353082,inlier
1,30.0,2,194740.0,9,13.0,2,11,0,4,1,0.0,0.0,40.0,39,1,0.142026,201,cd9d5349-22d4-43a9-aebb-68419083a4b1,1.358016,inlier
2,36.0,4,209629.0,11,9.0,2,7,0,4,1,0.0,0.0,40.0,39,1,0.146375,404,cd9d5349-22d4-43a9-aebb-68419083a4b1,0.996715,inlier


#### Step 2b. Inlier Evaluation

In [10]:
# Specify the inlier cases
inliers_indices = inliers[['.session', '.session_training_index']].values

# React to get the details of each case
results = t.react(
    case_indices=inliers_indices, 
    preserve_feature_values=df.columns.tolist(), 
    leave_case_out=True, 
    details=details
)

# Extract the global and local case feature residual convictions
inlier_global_case_feature_residual_convictions = pd.DataFrame(
    results['explanation']['global_case_feature_residual_convictions']
)[df.columns.tolist()]

inlier_local_case_feature_residual_convictions = pd.DataFrame(
    results['explanation']['local_case_feature_residual_convictions']
)[df.columns.tolist()]

In [11]:
plot_anomalies(inliers, inlier_global_case_feature_residual_convictions)

#### Step 2c. Inlier Inspection

This chart is viewed similarly to the outliers chart, however we are now examining Cases that are too similar. We can see that for several Cases, there are a few features that show high `feature_residual_conviction`, indicating that that feature can very easily predicted using the other features. However, this could also just be an indicator of high correlation between certain variables. In our Cases, we do not see a strong case of high `feature_residual_conviction` across a majority of features, thus it is likely that these points are not inliers. In many cases, especially with well known datasets like `Adult`, true inliers are rare as they are often the result of purposeful actions, while outliers may exist more naturally.

# Section 3. Next Steps

This recipe showed us how to detect and inspect anomalous cases. There are many courses of action to take on anomalous data, and Recipe `4-audit_edit.ipynb` shows us how we can take edit these data points in our Trainee without having to retrain the Trainee.

# Section 4: Appendix

Here we detail some additional examples methods of diving deeper into the possible anomalies detected above/

In [12]:
# Helper functions
def case_explain_residuals_ratio(anomalous_df, global_case_feature_residual_convictions, local_case_feature_residual_convictions, case_ind, num_features=5):
    case = anomalous_df.iloc[case_ind, : ]
    case_num = case['.session_training_index']
    case_df = pd.concat([case, global_case_feature_residual_convictions.iloc[case_ind, :], local_case_feature_residual_convictions.iloc[case_ind, :]], axis=1)
    case_df = case_df.loc[df.columns]
    case_df.columns = ['values', 'global_case_feature_residual_convictions', 'local_case_feature_residual_convictions']
    case_df = case_df.sort_values(['global_case_feature_residual_convictions', 'local_case_feature_residual_convictions'], ascending=True)
    print(f'key features which caused case {case_num} to be anomalous: {case_df.head().index.tolist()}')
    print('')
    display(case_df.head())

def get_cases(anomalous_df, results, case_ind):
    case = anomalous_df.iloc[case_ind, :]
    inf_cases = pd.DataFrame(results['explanation']['influential_cases'][case_ind])
    bound_cases = pd.DataFrame(results['explanation']['boundary_cases'][case_ind])
    
    print('Original Case:')
    display(case)
    print('')
    
    print('Influential Cases:')
    display(inf_cases)
    print('')
    
    print('Boundary Cases:')
    display(bound_cases)
    print('')

We can print out a more detailed view of the Outlier Residual Conviction chart shown earlier.

In [13]:
# Print out the explanations for outliers
for i in range(2):
    case_explain_residuals_ratio(
        outliers,
        outlier_global_case_feature_residual_convictions,
        outlier_local_case_feature_residual_convictions,
        i
    )
    print('_____________')

key features which caused case 627 to be anomalous: ['capital-gain', 'education-num', 'age', 'sex', 'education']



,values,global_case_feature_residual_convictions,local_case_feature_residual_convictions
capital-gain,99999.0,0.020817,0.027992
education-num,9.0,0.430661,0.515108
age,28.0,0.559644,0.481048
sex,0,0.587617,0.499695
education,11,0.669893,0.640583


_____________
key features which caused case 556 to be anomalous: ['capital-gain', 'native-country', 'race', 'workclass', 'education']



,values,global_case_feature_residual_convictions,local_case_feature_residual_convictions
capital-gain,99999.0,0.029624,0.139733
native-country,30,0.221512,0.281040
race,1,0.244912,0.243521
workclass,6,0.430675,0.628726
education,14,0.635148,0.960807


_____________


## Influential and boundary cases

Influential and Boundary Cases may also provide additional clues into potentially anomalous data.

**`Definitions`:**

**`Boundary Cases`:** 
Boundary Cases are a generalization of counterfactuals, as they apply across all feature types, including continuous. These will return archetypes that have the highest gradient magnitude of the ratio of similar Context Feature values to maximally different Action Feature values. These can aid in uncovering and understanding causal relationships among the data. Boundary Cases may or may not also be Influential Cases, depending on the density and topology of the data. If they are not influential, then their influence weight will be 0.

In [14]:
# Helper function to print out the explanations
def case_explain_residuals_ratio(anomalous_df, global_case_feature_residual_convictions, local_case_feature_residual_convictions, case_ind, num_features=5):
    case = anomalous_df.iloc[case_ind, : ]
    case_num = case['.session_training_index']
    case_df = pd.concat([case, global_case_feature_residual_convictions.iloc[case_ind, :], local_case_feature_residual_convictions.iloc[case_ind, :]], axis=1)
    case_df = case_df.loc[df.columns]
    case_df.columns = ['values', 'global_case_feature_residual_convictions', 'local_case_feature_residual_convictions']
    case_df = case_df.sort_values(['global_case_feature_residual_convictions', 'local_case_feature_residual_convictions'], ascending=True)
    print(f'key features which caused case {case_num} to be anomalous: {case_df.head().index.tolist()}')
    print('')
    display(case_df.head())

In [15]:
# Print influential and boundary case outliers
for i in range(2):
    get_cases(outliers, results, i)
    print('_____________')

Original Case:


age                                                                28.0
workclass                                                             4
fnlwgt                                                         126060.0
education                                                            11
education-num                                                       9.0
marital-status                                                        2
occupation                                                            1
relationship                                                          5
race                                                                  4
sex                                                                   0
capital-gain                                                    99999.0
capital-loss                                                        0.0
hours-per-week                                                     36.0
native-country                                                  


Influential Cases:


,hours-per-week,capital-loss,.session,.session_training_index,fnlwgt,.influence_weight,workclass,occupation,age,marital-status,education-num,capital-gain,target,education,race,sex,relationship,native-country
0,40,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,201,194740,0.716315,2,11,30,2,13,0,1,9,4,1,0,39
1,50,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,859,158688,0.041751,2,11,41,2,13,0,1,9,4,1,0,39
2,45,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,584,126584,0.037474,2,4,34,2,13,0,1,9,4,1,0,39
3,40,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,86,256997,0.035426,2,3,39,2,13,0,1,9,4,1,0,39
4,40,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,264,186934,0.024922,6,12,35,2,13,0,1,9,4,1,0,39
5,45,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,721,159773,0.022316,2,11,30,2,13,0,0,9,4,1,0,39
6,40,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,282,173938,0.019268,4,3,47,2,13,0,1,9,4,1,0,39
7,40,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,323,222011,0.018722,1,6,45,2,13,0,1,9,4,1,0,39
8,40,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,211,180881,0.017845,4,4,52,2,13,0,1,9,4,1,0,39
9,40,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,441,177828,0.016687,2,10,30,5,13,0,1,9,4,1,1,39



Boundary Cases:


...


Influential Cases:


,hours-per-week,capital-loss,.session,.session_training_index,fnlwgt,.influence_weight,workclass,occupation,age,marital-status,education-num,capital-gain,target,education,race,sex,relationship,native-country
0,40,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,624,181091,0.718927,2,11,30,2,13,0,1,9,4,1,0,39
1,50,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,859,158688,0.039596,2,11,41,2,13,0,1,9,4,1,0,39
2,40,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,86,256997,0.037405,2,3,39,2,13,0,1,9,4,1,0,39
3,45,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,584,126584,0.035741,2,4,34,2,13,0,1,9,4,1,0,39
4,40,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,264,186934,0.024888,6,12,35,2,13,0,1,9,4,1,0,39
5,45,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,721,159773,0.021721,2,11,30,2,13,0,0,9,4,1,0,39
6,40,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,323,222011,0.019295,1,6,45,2,13,0,1,9,4,1,0,39
7,40,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,282,173938,0.018832,4,3,47,2,13,0,1,9,4,1,0,39
8,40,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,211,180881,0.017475,4,4,52,2,13,0,1,9,4,1,0,39
9,43,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,494,269792,0.016945,1,1,44,2,13,0,1,9,4,1,0,39



Boundary Cases:


,hours-per-week,capital-loss,.session,.session_training_index,fnlwgt,.influence_weight,workclass,age,occupation,marital-status,familiarity_conviction_addition,education-num,education,target,capital-gain,race,sex,relationship,native-country
0,50,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,138,119986,0.000000,5,61,12,2,80.814567,13,9,1,0,4,1,0,39
1,50,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,355,184018,0.000000,4,43,10,2,55.265302,12,7,1,0,4,1,0,39
2,40,0,cd9d5349-22d4-43a9-aebb-68419083a4b1,441,177828,0.016366,2,30,10,5,33.601496,13,9,1,0,4,1,1,39



_____________
